In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [32]:
# send to GPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [44]:
class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 128), 
            nn.LeakyReLU(0.1),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        return self.disc(x)

    
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim), # dims are 28 x 28 pixels and 1 colour channel
            nn.Tanh(),
        )

    def forward(self, x):
        return self.gen(x)



In [49]:
# Hyperparameters 
# GANNS are very sensitive to changes in the hyperparameters

lr = 3e-4
z_dim = 64 
image_dim = 28 * 28 * 1
batch_size = 32
epochs = 50


In [46]:
disc = Discriminator(image_dim).to(device=device)
gen = Generator(z_dim,image_dim).to(device=device)


fixed_noise = torch.randn((batch_size,z_dim)).to(device=device)
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

dataset = datasets.MNIST(root='dataset/', transform=transform, download=True)

loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

opt_disc = optim.Adam(disc.parameters(), lr = lr)
opt_gen = optim.Adam(gen.parameters(), lr = lr)

criterion = nn.BCELoss()

In [47]:
writer_fake = SummaryWriter(f'runs/GAN_MNIST/fake')
writer_real = SummaryWriter(f'runs/GAN_MNIST/real')
step = 0

In [51]:
# Traning loop:

for epoch in range(epochs):
    for batch_idx, (real, _) in enumerate(loader): # usually (real, _) contain the images and the labels of the data
        real = real.view(-1, 784).to(device=device)
        batch_size = real.shape[0]

        ## Train Discriminator: max log(D(real)) + log(1 - D(G(z))
        noise = torch.randn(batch_size, z_dim).to(device=device)
        fake = gen(noise)

        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))

        lossD = lossD_real - lossD_fake / 2 

        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        ## Train Generator: min log(1 - D(G(z))) <--> max log(D(G(z)))

        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()

        opt_gen.step()

        if batch_idx == 0:
            print(
                f'Epoch [{epoch}/{epochs}] \ '
                f'Loss D: {lossD:.4f}, Loss G: {lossG:.4f}'

            )
        
            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)

                img_grid_fake = torchvision.utils.make_grid(fake, normalize = True)
                img_grid_real = torchvision.utils.make_grid(data, normalize = True)

                writer_fake.add_image(
                    'Mnist Fake Images', img_grid_fake, global_step=step
                    
                )

                writer_real.add_image(
                    'Mnist Real Images', img_grid_real, global_step=step

                )

                step += 1





Epoch [0/50] \ Loss D: -50.0000, Loss G: 0.0000
Epoch [1/50] \ Loss D: -50.0000, Loss G: 0.0000


KeyboardInterrupt: 